# System Versions

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.is_available())


2.6.0+cu118
0.21.0+cu118
True


In [2]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU Details:", gpus)
    print("TensorFlow is using the GPU")
else:
    print("No GPU detected")


Num GPUs Available: 1
GPU Details: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow is using the GPU


# Data Augmentation

In [3]:
import albumentations as A
import cv2
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [4]:
# Augmentations use paraller processing,,,

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=30, p=0.5),
    A.GaussianBlur(blur_limit=3, p=0.2),
], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))


In [5]:
def load_yolo_labels(label_path):
    if not os.path.exists(label_path):
        return []
    with open(label_path, "r") as f:
        return [list(map(float, line.strip().split())) for line in f.readlines()]

In [6]:
def save_yolo_labels(label_path, labels):
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")

In [7]:
def augment_image(image_path, label_path, output_dir):
    image = cv2.imread(image_path)
    if image is None:
        return


In [8]:
def augment_image(image_path, label_path, output_dir):
    image = cv2.imread(image_path)
    if image is None:
        return

    h, w = image.shape[:2]
    labels = load_yolo_labels(label_path)
    
    if not labels:
        return

    bboxes = [[x, y, bw, bh] for _, x, y, bw, bh in labels]
    class_labels = [label[0] for label in labels]

    for i in range(1):  # 3 augmentations per image
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_img = augmented["image"]
        aug_bboxes = augmented["bboxes"]

        new_labels = [[cls] + list(bbox) for bbox, cls in zip(aug_bboxes, class_labels)]

        output_image_path = os.path.join(output_dir, "images", f"aug_{i}_{os.path.basename(image_path)}")
        output_label_path = os.path.join(output_dir, "labels", f"aug_{i}_{os.path.basename(label_path)}")
        
        cv2.imwrite(output_image_path, aug_img)
        save_yolo_labels(output_label_path, new_labels)


In [9]:
def process_image(image_file):
    image_path = os.path.join(r"F:\camera\accident detection\dataset\train\images", image_file)
    label_path = os.path.join(r"F:\camera\accident detection\dataset\train\labels", image_file.replace(".jpg", ".txt").replace(".png", ".txt"))
    augment_image(image_path, label_path, r"F:\camera\accident detection\dataset\train")


In [10]:
if __name__ == "__main__":
    os.makedirs(r"F:\camera\accident detection\dataset\train\images", exist_ok=True)
    os.makedirs(r"F:\camera\accident detection\dataset\train\labels", exist_ok=True)

    image_files = [f for f in os.listdir(r"F:\camera\accident detection\dataset\train\images") if f.endswith((".jpg", ".png"))]

    with ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(process_image, image_files), total=len(image_files)))

    print("Fast Albumentations augmentation completed!")


100%|███████████████████████████████████████████████████████████████████████████| 12272/12272 [01:56<00:00, 105.04it/s]

Fast Albumentations augmentation completed!


# Model Training

# Phase I

In [2]:
from ultralytics import YOLO

In [3]:
model = YOLO('yolov8s.pt')

In [4]:
data_yaml = r'F:\camera\accident detection\dataset\data.yaml'

In [ ]:
model.train(data=data_yaml, epochs=10, batch=32, augment=False, imgsz=640, workers=2)

New https://pypi.org/project/ultralytics/8.3.98 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.94  Python-3.10.6 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=F:\camera\accident detection\dataset\data.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, emb

train: Scanning F:\camera\accident detection\dataset\train\labels.cache... 24538 images, 48 backgrounds, 0 corrupt: 100


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning F:\camera\accident detection\dataset\valid\labels.cache... 2319 images, 1 backgrounds, 0 corrupt: 100%|██


Plotting labels to F:\camera\runs\detect\train12\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to F:\camera\runs\detect\train12
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      6.71G      1.955      18.04      2.124         36        640:   1%|          | 5/767 [00:45<1:50:44,  

# Phase II (*10 - 20 Epochs*)

In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO(r"F:\camera\accident detection\runs\detect\train\weights\best.pt")

In [3]:
data_yaml = r'F:\camera\accident detection\dataset\data.yaml'

In [4]:
model.train(data=data_yaml, epochs=10, batch=16, augment=False, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.98 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.94  Python-3.10.6 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: task=detect, mode=train, model=F:\camera\accident detection\runs\detect\train\weights\best.pt, data=F:\camera\accident detection\dataset\data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnos

train: Scanning F:\camera\accident detection\dataset\train\labels.cache... 24538 images, 48 backgrounds, 0 corrupt: 100


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


F:\camera\.camera_venv\lib\site-packages\albumentations\check_version.py:107: UserWarning: Error fetching version info The read operation timed out
  data = fetch_version_info()
val: Scanning F:\camera\accident detection\dataset\valid\labels.cache... 2319 images, 1 backgrounds, 0 corrupt: 100%|██


Plotting labels to F:\camera\runs\detect\train15\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to F:\camera\runs\detect\train15
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       2319       3532      0.574      0.586      0.538      0.432



10 epochs completed in 5.791 hours.
Optimizer stripped from F:\camera\runs\detect\train15\weights\last.pt, 22.5MB
Optimizer stripped from F:\camera\runs\detect\train15\weights\best.pt, 22.5MB

Validating F:\camera\runs\detect\train15\weights\best.pt...
Ultralytics 8.3.94  Python-3.10.6 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
Model summary (fused): 72 layers, 11,128,293 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [01:09


                   all       2319       3532      0.576      0.586      0.538      0.432
            NoAccident        187       1016      0.532      0.752      0.601       0.31
                   car         53        281      0.184      0.242      0.148     0.0621
                  mild         15         19      0.408      0.474      0.407      0.315
              moderate        536        543      0.881      0.943      0.952      0.895
           motor cycle          3         15       0.48      0.125      0.127     0.0468
                severe       1583       1658      0.968      0.979      0.992      0.965
Speed: 0.4ms preprocess, 15.8ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to F:\camera\runs\detect\train15


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001D7B3204C70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
 

# Phase III (*20 - 25 Epochs*)

# Pending

In [1]:
from ultralytics import YOLO

In [2]:
data_yaml = r'F:\camera\accident detection\dataset\data.yaml'

In [3]:
model = YOLO(r"F:\camera\accident detection\runs\detect\train2\weights\best.pt")

In [4]:
model.train(data=data_yaml, epochs=5, batch=16, augment=False, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.101 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.94  Python-3.10.6 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: task=detect, mode=train, model=F:\camera\accident detection\runs\detect\train2\weights\best.pt, data=F:\camera\accident detection\dataset\data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agno

100%|█████████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [00:04<00:00, 1.36MB/s]


AMP: checks passed 


train: Scanning F:\camera\accident detection\dataset\train\labels.cache... 24538 images, 48 backgrounds, 0 corrupt: 100


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning F:\camera\accident detection\dataset\valid\labels.cache... 2319 images, 1 backgrounds, 0 corrupt: 100%|██

KeyboardInterrupt



# Phase IV (*30 - 40 Epochs*)

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO(r"F:\camera\accident detection\runs\detect\train30\weights\best.pt")

In [ ]:
model.train(data=data_yaml, epochs=10, batch=16, augment=False, imgsz=640)

# Phase V (*40 - 50 Epochs*)

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO(r"F:\camera\accident detection\runs\detect\train30\weights\best.pt")

In [ ]:
model.train(data=data_yaml, epochs=10, batch=16, augment=False, imgsz=640)

# Demo

In [9]:
import cv2
from ultralytics import YOLO

model = YOLO(r"F:\camera\accident detection\runs\detect\train2\weights\best.pt")
# model = YOLO(r"F:\camera\accident detection\delete\best.pt")

class_names = {
    0: "Accident",
    1: "NoAccident",
    2: "Car",
    3: "Mild",
    4: "Moderate",
    5: "Motor Cycle",
    6: "Severe"
}

# video_path = "demo_accident.mp4"
video_path = r"F:\camera\accident detection\delete\acc.mp4"

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

display_width = 800  # Adjust as needed

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    aspect_ratio = width / height
    new_height = int(display_width / aspect_ratio)
    frame = cv2.resize(frame, (display_width, new_height))

    results = model(frame)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box
            conf = float(box.conf[0])  # Confidence score
            cls = int(box.cls[0])  # Class ID
            label = f"{class_names.get(cls, 'Unknown')} ({conf:.2f})"

            color = (0, 0, 255) if cls == 0 else (0, 255, 0) if cls == 1 else (255, 0, 0)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Accident Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 640x384 (no detections), 12.4ms
Speed: 1.4ms preprocess, 12.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 11.9ms
Speed: 2.6ms preprocess, 11.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 11.8ms
Speed: 2.2ms preprocess, 11.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 11.8ms
Speed: 1.6ms preprocess, 11.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 NoAccident, 11.7ms
Speed: 1.9ms preprocess, 11.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 NoAccident, 11.7ms
Speed: 1.8ms preprocess, 11.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 11.8ms
Speed: 1.8ms preprocess, 11.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 11.9ms
Speed: 1.8ms preprocess, 11.9ms inferen

In [3]:
import cv2
video_path = "demo_accident.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()
